## Chapter 3 평가

앞에서는 모델 예측 성능의 평가를 위해 정확도(Accuracy)를 이용했다. 하지만 이 밖에도 여러 가지 방법으로 예측 성능을 평가할 수 있다. <br/> 성능 평가 모델은 일반적으로 모델이 분류냐 회귀냐에 따라 다르다. <br/> 분류의 성능 평가 지표는 다음과 같다.
- 정확도(Accuracy)
- 오차행렬(Confusion Matrix0
- 정밀도(Precision)
- 재현율(Recall)
- F1 스코어
- ROC AUC

분류의 경우 결정 클래스 값 종류의 유형에 따라 이진 분류와 선택 분류로 나뉠 수 있다. 